In [ ]:
#蝦皮總上架ID

import tkinter as tk
import requests
import json
import xlrd
from googlesheet import delsheet
from googlesheet import getsheet
from googlesheet import writesheet
xlrd.Book.encoding = "utf8" #设置编码
data = xlrd.open_workbook(r"productsID.xlsx")
table1 = data.sheet_by_index(0) #取第一张工作簿

def updateSPID():
    webidAry = []
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    for i in range(200):
        url = "https://shopee.tw/api/v2/search_items/?by=pop&limit=100&match_id=2019696&newest="+str(i*100) +"&order=desc&page_type=shop"
        gotrq = requests.get(url,headers = my_headers)
        loadjson = json.loads(gotrq.text)
        if len(loadjson["items"]) == 0:
            break
        for ct in range(len(loadjson["items"])):
            webidAry.append(loadjson["items"][ct]["itemid"])

    #比對新增的WEBID
    #GOTData = getsheet('商品ID!I:I') #獲取試算表所有資料
    #IDAry = [] #已上的ID
    IDAry = table1.col_values(8)
    errorAry = [] #找到的未上ID
    #for i in range(len(GOTData)): #提取資料
    #    try:
    #        IDAry.append(GOTData[i][0])
    #    except:
    #        IDAry.append("")
    for i in range(len(webidAry)): #找到未上的ID
        try:
            IDAry.index(str(webidAry[i]))
        except:
            errorAry.append(webidAry[i])
    
    output.insert(1.0,"查詢完成\n")
    for i in range(len(errorAry)):
        output.insert(1.0,str(errorAry[i])+"\n")


        

def clickbtn():
    try:
        output.delete('1.0',tk.END)
    except:
        print("start")
    updateSPID()
    
    
gui=tk.Tk()
gui.title("查蝦皮新上架")
output= tk.Text()
output.pack()
btn = tk.Button(gui, text ="執行", command = clickbtn)
btn.pack(anchor='center', expand='yes')
gui.mainloop()

In [10]:
#蝦皮庫存
import xlrd
import time
import tkinter as tk
import tkinter.filedialog as filedialog
import os
from googlesheet import delsheet
from googlesheet import getsheet
from googlesheet import writesheet
import xlrd


def browser():
    fname = filedialog.askopenfilename(initialdir= os.getcwd(),filetypes = (("Excel","*.xlsx"),("all files","*.*")))
    global filepath
    filepath =  fname  # 返回文件全路径
    pathinput.delete('0',tk.END)
    pathinput.insert(0,filepath)
    
    #print(filedialog.askdirectory())  # 返回目录路径
    
def shopeeExcel(path):
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(path)
    table = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table.nrows #取总行数
    prdid = []
    for row in range(rows_count):#無款式
        if table.cell(row,0).value:
            try:
                prdid.append({"ID":str(int(table.cell(row,0).value)),"stock":str(int(table.cell(row,6).value))})
            except:
                a = "a"
    for i in range(20):
        col = 5*i
        for row in range(rows_count): #商品款式
            try:
                prdid.append({"ID":str(int(table.cell(row,8+col).value)),"stock":str(int(table.cell(row,12+col).value))})
            except:
                a = "b"
    return prdid
def run():
    print(filepath)
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(r"productsID.xlsx")
    table1 = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table1.nrows #取总行数
    #getidlist = getsheet("商品ID!I:J")
    oklist = shopeeExcel(filepath) #讀取EXCEL
    #提取sheet 資料
    sprdid = table1.col_values(8)
    smodelid = table1.col_values(9)
    #for i in range(len(getidlist)):
    #    try:
    #        sprdid.append(getidlist[i][0])
    #    except:
    #        sprdid.append("")
    #    try:
    #        smodelid.append(getidlist[i][1])
    #    except:
    #        smodelid.append("")
        
    #尋找並寫入sheet
    wrval = []
    noidlist = []
    wriso = [] #更新標題
    for i in range(rows_count):
        wriso.append([table1.cell_value(i,0),table1.cell_value(i,1),table1.cell_value(i,2)])

    for i in range(len(getidlist)):
        wrval.append([""])
    for i in range(len(oklist)):

        try:
            findrow = sprdid.index(oklist[i]["ID"])
            wrval[findrow] = [oklist[i]["stock"]]
        except:
            try:
                findrow = smodelid.index(oklist[i]["ID"])
                wrval[findrow] = [oklist[i]["stock"]]
            except:
                noidlist.append(oklist[i]["ID"])

    wrval[0] = ["Shopee"+ time.strftime("%m/%d", time.localtime())]
    delsheet("商品ID!R:R")
    writesheet("商品ID!A1",wriso)
    writesheet("商品ID!R1",wrval)

gui=tk.Tk()
gui.title("更新蝦皮庫存")
gui.geometry("450x50")
pathinput = tk.Entry(width=50)
pathinput.pack(side="left")
btn2 = tk.Button(gui, text ="瀏覽", command = browser)
btn2.pack(side="left")
btn = tk.Button(gui, text ="執行", command = run)
btn.pack(side="left")
gui.mainloop()



In [ ]:
#蝦皮庫存表整理
import xlrd
import xlwt
xlrd.Book.encoding = "utf8" #设置编码
data = xlrd.open_workbook(r"C:\Users\Owner\Downloads\items_2019_02_20.xlsx")
table1 = data.sheet_by_index(0) #取第一张工作簿
rows_count = table1.nrows #取总行数

nwb = xlwt.Workbook()# 新增EXCEL
nsheet = nwb.add_sheet('蝦皮款式')#新SHEET
nsheet.write(0,0,'商品名稱')
nsheet.write(0,1,'商品ID')
nsheet.write(0,2,'商品款式條碼')
nsheet.write(0,3,'國際條碼')
nsheet.write(0,4,'款式名稱')
nsr = 1 #寫入第二列

style1 = xlwt.XFStyle() #設定樣式
al = xlwt.Alignment()
al.horz = 0x03 #向右對齊
style1.alignment = al

for r in  range(20): #20個選項都跑一次 r為第幾個款式 (欄位)
    col = 8+5*r #下一個款式為下五欄
    for i in range(3,rows_count):#列述迴圈
        if table1.cell_type(i,col): #類型不是empty的
            
            nsheet.write(nsr,0,table1.cell_value(i,2))#寫入讀取的資料 商品名
            nsheet.write(nsr,1,str(int(table1.cell_value(i,0))),style = style1)#寫入讀取的資料 商品ID
            nsheet.write(nsr,2,table1.cell_value(i,col),style = style1)#寫入讀取的資料 款式ID
            nsheet.write(nsr,3,table1.cell_value(i,col+1),style = style1)#寫入讀取的資料 ISO
            nsheet.write(nsr,4,table1.cell_value(i,col+2))#寫入讀取的資料 名稱
            nsr = nsr +1 #下一列
            #print(int(table1.cell_value(i,0)),table1.cell_value(i,col),table1.cell_value(i,col+1))#讀取值
        elif r == 0 and table1.cell_type(i,col) == 0: #列出無款式的商品
            nsheet.write(nsr,0,table1.cell_value(i,2))#寫入讀取的資料 商品名
            nsheet.write(nsr,1,str(int(table1.cell_value(i,0))),style = style1)#寫入讀取的資料 商品ID
            nsheet.write(nsr,3,table1.cell_value(i,1),style = style1)#寫入讀取的資料 ISO
            nsr = nsr +1 #下一列





nsheet.col(0).width= 10000 #欄位寬度
nsheet.col(1).width= 4000
nsheet.col(2).width= 4000
nsheet.col(3).width= 6000
nsheet.col(4).width= 4000
nwb.save('res.xls')

In [11]:
import xlrd
xlrd.Book.encoding = "utf8" #设置编码
data = xlrd.open_workbook(r"productsID.xlsx")
table1 = data.sheet_by_index(0) #取第一张工作簿
print(table1.col_values(14))

['松果', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '170927085242', '170927085242', '170927085242', '170927085242', '171102936281', '171102936281', '171102936281', '171102936281', '190123765346', '190123765346', '190123765346', '190123765346', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '160919038807', '160919038807', '160919038807', '160919038807', '160919038807', '', '', '190123933990', '190123933990', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '161223560770', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '161223496408', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '

In [ ]:
for i in range(20):
    print(i)